In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from eq_odds_postprocessing import EqOddsPostprocessing
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso
from random import sample

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## Huangrui's Dataset 

In [2]:
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
    
class Zindi(StandardDataset):
    """financial-inclusion-in-africa dataset.
    """

    def __init__(self, path, label_name='Y', favorable_classes=[1],  
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_drop=[],
                 features_to_keep=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        """See :obj:`RegressionDataset` for a description of the arguments."""
        

        df = pd.read_csv(path)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)

        super(Zindi, self).__init__(
            df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [3]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [4]:
dataset_orig_train= Zindi(path="./Huangrui/zindi/zindi_train1.csv",protected_attribute_names=['sex'],
                privileged_classes=[[1]])

#### Split into train, test and validation

In [5]:
experiments_info = {}
budget = 1
for K in range(1, 6):
    # dataset_orig_train= Zindi(path="./Huangrui/zindi/zindi_train{}.csv".format(K),protected_attribute_names=['sex'],
    #             privileged_classes=[[1]])
    dataset_orig_test= Zindi(path="./Huangrui/zindi/zindi_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    # #only use the budget% of the training data
    # dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)
    # Lasso linear classifier and predictions
    lmod = pickle.load(open('experiments/zindi'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features.copy()
    X_test[:,1] = [1]*X_test.shape[0]
    y_test = dataset_orig_test_pred.labels
    y_test_pred = lmod.predict(X_test)
    dataset_orig_test_pred.labels = (y_test_pred>0.5).reshape(-1,1)
    metric_test = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                                  unprivileged_groups, privileged_groups)
   
    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels), "Equal opportunity difference": metric_test["Equal opportunity difference"]}

Balanced accuracy = 0.5915
Statistical parity difference = -0.0214
Disparate impact = 0.5976
Average odds difference = -0.0089
Equal opportunity difference = -0.0168
Theil index = 0.1272
K = 1, budget = 1
The Error for the test dataset is 0.1273
The Equal opportunity difference for the test dataset is -0.01682
Balanced accuracy = 0.6628
Statistical parity difference = -0.0274
Disparate impact = 0.7144
Average odds difference = 0.0030
Equal opportunity difference = 0.0174
Theil index = 0.1059
K = 2, budget = 1
The Error for the test dataset is 0.1192
The Equal opportunity difference for the test dataset is 0.01739
Balanced accuracy = 0.6228
Statistical parity difference = -0.0352
Disparate impact = 0.5179
Average odds difference = -0.0281
Equal opportunity difference = -0.0517
Theil index = 0.1142
K = 3, budget = 1
The Error for the test dataset is 0.1175
The Equal opportunity difference for the test dataset is -0.05167
Balanced accuracy = 0.6182
Statistical parity difference = -0.0254


In [6]:
experiments_info

{'K = 1, budget = 1': {'Error': 0.12733843537414966,
  'Equal opportunity difference': -0.016819822776501486},
 'K = 2, budget = 1': {'Error': 0.11923485653560043,
  'Equal opportunity difference': 0.017388604253997797},
 'K = 3, budget = 1': {'Error': 0.11753453772582359,
  'Equal opportunity difference': -0.05167173252279633},
 'K = 4, budget = 1': {'Error': 0.11647183846971307,
  'Equal opportunity difference': -0.017234123564658005},
 'K = 5, budget = 1': {'Error': 0.1226354941551541,
  'Equal opportunity difference': -0.018752658895601126}}